In [23]:
!pip install langchain
!pip install openai
!pip install -U langchain-community
!pip install tiktoken
!pip install faiss-gpu
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trahu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trahu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trahu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trahu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-gpu


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ------------------------- -------------- 8.7/13.7 MB 48.8 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 50.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\trahu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import os
import zipfile


zip_path = "python-3.13-docs-text.zip"
extract_dir = "python_docs"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ ZIP extracted to:", extract_dir)


✅ ZIP extracted to: python_docs


In [25]:
from langchain.docstore.document import Document

documents = []

for root, _, files in os.walk("python_docs"):
    for file in files:
        if file.endswith(".txt"):
            with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                text = f.read()
                documents.append(Document(page_content=text, metadata={"source": file}))

print(f"📄 Loaded {len(documents)} text documents.")


📄 Loaded 507 text documents.


In [26]:
#split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

print(f"✂️ Total text chunks: {len(split_docs)}")


✂️ Total text chunks: 15919


In [27]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from getpass import getpass
import os

# Ask for API key safely
os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

# Generate embeddings
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


🔐 Enter your OpenAI API key:  ········


In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


C:\Users\trahu\AppData\Local\Temp\ipykernel_13252\2928076918.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    output_key="answer"  # ✅ tell it to store only the answer
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"  # ✅ same fix here
)


C:\Users\trahu\AppData\Local\Temp\ipykernel_13252\682399099.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
print("🤖 Ask me anything about Python docs! Type 'exit' to quit.\n")

while True:
    query = input("🧑 You: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("👋 Goodbye!")
        break

    result = conversation_chain({"question": query})

    print("\n🤖 Answer:\n" + result["answer"])

    print("\n📚 Sources:")
    for doc in result["source_documents"]:
        print(" -", doc.metadata["source"])
    
    print("\n---")


🤖 Ask me anything about Python docs! Type 'exit' to quit.



🧑 You:  can you give me if else function in java?


C:\Users\trahu\AppData\Local\Temp\ipykernel_13252\3500014405.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})



🤖 Answer:
I don't know.

📚 Sources:
 - controlflow.txt
 - compound_stmts.txt
 - programming.txt
 - controlflow.txt

---


🧑 You:  compare python and java programming language


...


🤖 Answer:
I don't have information on Java syntax.

📚 Sources:
 - controlflow.txt
 - controlflow.txt
 - controlflow.txt
 - itertools.txt

---


🧑 You:  do it for java



🤖 Answer:
I don't know.

📚 Sources:
 - controlflow.txt
 - 3.10.txt
 - programming.txt
 - compound_stmts.txt

---


🧑 You:  quit


👋 Goodbye!
